In [1]:
import pandas as pd, numpy as np
import sklearn
import scipy.sparse
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from scipy import sparse
import re
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss


In [2]:
train_data = pd.read_csv("./train_preprocessed_v1.csv") #generated via preprocessing.ipynb
test_data = pd.read_csv("./test_preprocessed_v1.csv")

In [3]:
label = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [4]:
#train_data["clean"] = 1-train_data[label].max(axis=1)
#label.insert(0,"clean")

In [5]:
train, validate = train_test_split(train_data, test_size=0.2)
training_text = train.comment_text
validation_text = validate.comment_text
test_text = test_data.comment_text

training_labels = train[label].values
validation_labels = validate[label].values

In [6]:
total_comments = training_text.size

In [7]:
training_text[0]
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000
mean,0.100000,0.008333,0.054167,0.008333,0.058333,0.020833
std,0.300627,0.091096,0.226819,0.091096,0.234862,0.143125
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [9]:
vec = TfidfVectorizer()
X_train_tfidf = vec.fit_transform(training_text)
X_valid_tfidf = vec.transform(validation_text)
y_test_tfidf = vec.transform(test_text)

In [10]:
X_train_tfidf,X_valid_tfidf

(<240x2828 sparse matrix of type '<class 'numpy.float64'>'
 	with 6348 stored elements in Compressed Sparse Row format>,
 <60x2828 sparse matrix of type '<class 'numpy.float64'>'
 	with 990 stored elements in Compressed Sparse Row format>)

In [18]:
def evaluate_score(Y_test,predict): 
    
    accuracy = accuracy_score(Y_test,predict)
    print("Accuracy : {}".format(accuracy*100))
    try : 
        loss = log_loss(Y_test,predict)
    except :
        loss = log_loss(Y_test,predict.toarray())
    print("Log_loss : {}".format(loss))

In [12]:
from sklearn.naive_bayes import MultinomialNB
clf = []
for i in range(6):
    clf.append(MultinomialNB())
    clf[i].fit(X_train_tfidf,training_labels[:,i])

# PREDICTION
predict = []
for i in range(6):
    predict.append(clf[i].predict(X_valid_tfidf))

predict = np.asarray(np.transpose(predict))
print(predict.shape)
print(validation_labels.shape)

(60, 6)
(60, 6)


In [19]:
evaluate_score(validation_labels,predict)

Accuracy : 88.33333333333333
Log_loss : 0.47780252512748134


In [16]:
from sklearn.naive_bayes import MultinomialNB
clf2 = []
for i in range(6):
    clf2.append(MultinomialNB())
    clf2[i].fit(X_train_tfidf,training_labels[:,i])

# PREDICTION
test_predict = []
for i in range(6):
    test_predict.append(clf[i].predict(y_test_tfidf))

test_predict = np.asarray(np.transpose(test_predict))
#print(test_predict.shape)


(100, 6)


In [20]:
#test_predict

In [22]:
subm = pd.read_csv('./sample_submission.csv')

submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(test_predict, columns = label)], axis=1)
submission.to_csv('submission.csv', index=False)